In [1]:
import numpy as np
import pandas as pd
import logging
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
from numpy.linalg import cholesky, det, lstsq,inv
logging.basicConfig(level=logging.INFO)
from scipy.optimize import minimize

In [2]:
# ref [2] eq(5)
def kernel_vec_gaussian_w_precision(xp,xq,p,q,lambda_diag,v,sigma_n):
    logger = logging.getLogger()
    logger.setLevel(level=logging.DEBUG)
    """
    xp: float array mx1 vector of input point p
    xq : float array
    lambda_ 1Xm represents the diagonal of the hyperparemter diagonal matrix lambda_
    v: hyperparemter
    sigma2_n : hyperparameter
    """
    #TODO check all inputes are doubles
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    exponent_ = -0.5*np.matmul(np.matmul((xp-xq).T,lambda_power_minus_1),(xp-xq))
    delta_pq = 1.0 if p==q else 0.0
    kernel = v**2 * np.exp(exponent)+delta_pq*sigma_n**2
    #logging
    logger.debug(f'lambda diag= {lambda_diag}')
    logger.debug(f'lambda_pow_minus_1 {lambda_power_minus_1}')
    logger.debug(f'term1 = -(xp-xq).T * lambda_pow_minus_1 = {term1}')
    logger.debug(f'exponent = {exponent_}')
    ######
    return kernel

In [3]:
# quick validation :kernel vs euclidean distance , comment out , time consuming
# dim = 100,5
# X = np.random.normal(loc=0,scale=1.0,size=dim)
# k = np.zeros(dim[0]**2)
# euc = np.zeros(dim[0]**2)
# idx = 0
# for i in np.arange(dim[0]):
#     for j in np.arange(dim[0]):
#         xp = X[i,].reshape(-1,1)
#         xq = X[j,].reshape(-1,1)
#         k[idx] = kernel(xp=xp,xq=xq,p=i,q=j,lambda_diag=[0.2,0.1,0.3,0.4,0.5],sigma_n=0.05,v=1.0)
#         euc[idx] = euclidean(u=xp,v=xq)
#         idx = idx + 1
# pearsonr(x=k,y=np.reciprocal(euc+0.01))
# if corr is +ve and significant, then we are good

In [4]:
# Note: in http://krasserm.github.io/2018/03/19/gaussian-processes/
# in the implementatoin of Gaussian kernel , the sum of the two squared terms is broadcast sum
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.add.html
# https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
# why ? as in first term , reshape(-1,1) is used , then the row sum shape is converted from (m,) to (m,1)
# then it is summed with array of shape (n,) then shapes are not equal, even if m==n , then the second dimensions 
# are not equal
import numpy as np
def kernel_gaussian(X1, X2, l=1.0, sigma_f=1.0):
    ''' Isotropic squared exponential kernel. Computes a covariance matrix from points in X1 and X2. Args: X1: Array of m points (m x d). X2: Array of n points (n x d). Returns: Covariance matrix (m x n). '''
    # the sume is broad cast 
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

X1 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X2 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X1_sum= np.sum(X1**2, 1).reshape(-1,1)
X2_sum= np.sum(X2**2, 1)
X2_sum_2= np.sum(X2**2, 1).reshape(-1,1)
tot = X1_sum + X2_sum
print((X1_sum + X2_sum).shape)
print((X1_sum + X2_sum_2).shape)

(10, 10)
(10, 1)


In [5]:
# ref [2] eq(5)
def kernel_mtx_gaussian_w_precision(X1,X2,lambda_diag,v,sigma_n):
    ## from http://krasserm.github.io/2018/03/19/gaussian-processes/
    ## sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    
    logger = logging.getLogger()
    logger.setLevel(level=logging.INFO)
    assert X1.shape[1] == X2.shape[1]
    assert X1.shape[1] == len(lambda_diag)
    
    
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    # quadratic terms are scale by inverse precision (variance) , precision = lambda (Captial case)
    X1_sq_sum_scaled = np.sum(a=np.matmul(X1**2,lambda_power_minus_1),axis=1)
    X2_sq_sum_scaled = np.sum(a=np.matmul(X2**2,lambda_power_minus_1),axis=1)
    X1_matmul_X2_T_scaled = np.matmul(np.matmul(X1,lambda_power_minus_1),X2.T)
    
    sqdist_scaled = X1_sq_sum_scaled.reshape(-1,1)+X2_sq_sum_scaled-2*X1_matmul_X2_T_scaled
    sigma_n_sq = np.zeros((X1.shape[0],X2.shape[0]))
    if X1.shape[0] == X2.shape[0]: #same data-set , or at least comparable
        sigma_n_sq = np.diag(np.repeat(sigma_n**2,X1.shape[0]))
        
    kernel = v**2 * np.exp(-0.5*sqdist_scaled)+sigma_n_sq
    logger.debug(f'sqdist_scaled = {sqdist_scaled}')
    return kernel


M = 10
N = 8
D = 3
X1 = np.random.normal(loc = 10, scale  = 2, size = (M,D))
X2 = np.random.normal(loc = 5, scale  = 1, size = (N,D))
lambda_diag = np.random.normal(loc = 1, scale = 0.01, size = D)
v = 1
sigma_n = 0.05

k = kernel_mtx_gaussian_w_precision(X1,X1,lambda_diag,v,sigma_n)
np.diag(k)

array([1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025,
       1.0025, 1.0025])

In [6]:
# packing theta
def kernel_mtx_gaussian_w_precision_fn(theta,X1,X2):
    assert X1.shape[1] == X2.shape[1]
    D = X1.shape[1]
    lambda_diag=theta[0:D]
    v= theta[D]
    sigma_n = theta[D+1]
    return kernel_mtx_gaussian_w_precision(X1,X2,lambda_diag,v,sigma_n)

In [7]:
def kernel_gaussian_fn(theta,X1,X2):
    assert X1.shape[1] == X2.shape[1]
    l = theta[0]
    sigma_f = theta[1]
    return kernel_gaussian(X1, X2, l, sigma_f)

In [8]:
# calc neg_LL (ignore constant terms , deosn't affect optimization results)
def neg_ll(theta,X,t,kernel_fn):
    assert X.shape[0]==t.shape[0]
    K = kernel_fn(theta,X,X)
    neg_ll_tot = 0
    for j in np.arange(t.shape[1]):    
        neg_ll_j = np.log(det(K))+t[:,j].T.dot(inv(K).dot(t[:,j]))
        neg_ll_tot = neg_ll_tot+neg_ll_j
    return neg_ll_tot

t = np.random.normal(loc=5,scale = 1,size = (M,1))
theta_0 = np.random.normal(loc=2,scale = 1,size=X1.shape[1]+2)
neg_ll(X=X1,t=t,kernel_fn=kernel_mtx_gaussian_w_precision_fn,theta=theta_0)

65.20351007416716

In [9]:
# first trial for minimization
minimize(fun=neg_ll,x0=theta_0,args=(X1,t,kernel_mtx_gaussian_w_precision_fn))

      fun: 15.35723797945462
 hess_inv: array([[ 3.07453542e-01,  1.80094333e+00,  1.27527574e+00,
         1.16019856e+00,  2.79225127e-02],
       [ 1.80094336e+00,  4.12654526e+01,  1.52867817e+01,
         3.04558647e+00, -9.67237500e-02],
       [ 1.27527576e+00,  1.52867817e+01,  7.33060558e+00,
         3.92081993e+00,  6.45276123e-02],
       [ 1.16019856e+00,  3.04558647e+00,  3.92081993e+00,
         4.91591508e+00,  1.58584747e-01],
       [ 2.79225127e-02, -9.67237500e-02,  6.45276123e-02,
         1.58584747e-01,  2.28735542e-02]])
      jac: array([-1.12056732e-05, -8.10623169e-06,  1.00135803e-05,  7.67707825e-05,
       -1.08146667e-03])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 1354
      nit: 74
     njev: 192
   status: 2
  success: False
        x: array([ 8.40068101e+03,  1.57480286e+03,  7.07204941e+03,  5.67060044e+00,
       -9.20966258e-01])

### Note : 
With default method (not show or known yet) , optimization doesn't converge. Try with explcitly defined methods

In [10]:
minimize(fun=neg_ll,x0=theta_0,args=(X1,t,kernel_mtx_gaussian_w_precision_fn),method="L-BFGS-B")

      fun: 15.426999965247807
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.20268248e-05,  2.82440737e-05,  1.33226763e-05, -7.40740802e-05,
        1.75823800e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 516
      nit: 55
   status: 0
  success: True
        x: array([ 5.18687352e+03,  3.53176184e+03,  4.63838946e+03,  5.64213874e+00,
       -9.29346919e-01])

### Note
with L-BFGS-B , the optimization converges and success. Hence, see TODO

### Illustrative example in [2] Section 3, Dynamics GP Model Identification 

In [11]:
import pandas as pd
train_data = pd.read_csv('../data/train_data2019-10-01T09:12:32.226141.csv')
train_data.head()
X = train_data[['x','x_dot','F']].values
D_x = X.shape[1]
t = train_data[['x_prime','x_dot_prime']].values
t_prime = t-np.mean(t,axis=0) # to satisfy E(t) = 0 condition , used for ll calculations
N = X.shape[0]
N_t = 50 # int(np.floor(0.8*X.shape[0]))

In [12]:
n_trials = 10

In [13]:
# Model 1: with Kernal in [2] eqn(5), gaussian with precision
min_obj_fn_1 = np.Inf
theta_min_1 = None
res_min_1 = None
logger = logging.getLogger()
for i in np.arange(n_trials):
    theta_0 = np.random.normal(loc=2,scale = 1,size=D_x+2)
    res = minimize(fun=neg_ll,x0=theta_0,args=(X[:N_t],t_prime[:N_t],kernel_mtx_gaussian_w_precision_fn)\
                             ,method='L-BFGS-B')
    logger.info(f'minimize results at train index = {i} = \n{res}')
    if res['fun'] < min_obj_fn_1:
        res_min_1= res
        min_obj_fn_1 = res['fun']
        theta_min_1 = res['x']
res

INFO:root:minimize results at train index = 0 = 
      fun: -502.84640527423
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.17197976, -0.06907044, -0.12272494,  0.0413138 , -0.06933192])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 714
      nit: 40
   status: 0
  success: True
        x: array([ 3.80352829e+01,  4.67392363e+01,  7.75338865e+01, -3.09417322e+00,
       -1.28534036e-02])
INFO:root:minimize results at train index = 1 = 
      fun: -523.3607130580577
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.15122623, -0.22690756, -0.28122713, -0.02656861,  0.14352963])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 642
      nit: 42
   status: 0
  success: True
        x: array([5.43041841e+01, 8.05018718e+01, 1.58681050e+02, 3.49093729e+00,
       1.52129489e-02])
/home/mbaddar/mbaddar/my_projects/phd/implementation/gprl/gprl_venv/lib/python3.7/site-packages/ipykernel_l

      fun: -498.02310754697055
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.67922598e-02, -4.01314537e-02, -3.51906237e-01,  6.67120048e+00,
        8.82966128e+02])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 306
      nit: 18
   status: 0
  success: True
        x: array([ 9.53665443e+01,  4.45475814e+01,  7.43283025e+01,  5.98366039e+00,
       -1.12601205e-02])

In [14]:
#Method 2 , ordinary Gaussian Kernel
n_trials = 10
min_obj_fn_2 = np.Inf
theta_min_2 = None
res_min_2 = None
logger = logging.getLogger()
for i in np.arange(n_trials):
    theta_0 = np.random.normal(loc=2,scale = 1,size=2)
    res = minimize(fun=neg_ll,x0=theta_0,args=(X[:N_t],t_prime[:N_t],kernel_gaussian_fn)\
                             ,method='L-BFGS-B')
    logger.info(f'minimize results at train index = {i} = \n{res}')
    if res['fun'] < min_obj_fn_2:
        res_min_2 = res
        min_obj_fn_2 = res['fun']
        theta_min_2 = res['x']
res

INFO:root:minimize results at train index = 0 = 
      fun: -364.07006704236153
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.03140599, 0.13594672])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 78
      nit: 12
   status: 0
  success: True
        x: array([3.32188542, 1.91267712])
INFO:root:minimize results at train index = 1 = 
      fun: -364.0713846408736
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.74581408, -0.16929107])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 66
      nit: 10
   status: 0
  success: True
        x: array([3.32699845, 1.92231163])
/home/mbaddar/mbaddar/my_projects/phd/implementation/gprl/gprl_venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/home/mbaddar/mbaddar/my_projects/phd/implementation/gprl/gprl_venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarni

      fun: -364.0686501336685
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.36233211, -0.22944846])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 63
      nit: 7
   status: 0
  success: True
        x: array([3.32465381, 1.91227627])

### in-sample prediction

In [15]:
def y_pred(x,X,t,kernel_fn,theta):
    k = kernel_fn(theta,x,X)
    K = kernel_fn(theta,X,X)
    y_mean = np.matmul(k,np.matmul(inv(K),t))
    return y_mean

In [16]:
sample_idx = np.arange(N_t,(N_t+100))
x = X[sample_idx,:]
x.shape

(100, 3)

In [17]:
t_prime[sample_idx,:].shape

(100, 2)

In [18]:
y_pred_ = y_pred(x,X[:N_t],t_prime[:N_t],kernel_mtx_gaussian_w_precision_fn,theta_min_1)
y_pred_.shape

(100, 2)

In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [20]:
rmse_y1 = sqrt(mean_squared_error(y_true = t_prime[sample_idx,0],y_pred=y_pred_[:,0]))
rmse_y1

0.007448532246993616

In [21]:
rmse_y2 = sqrt(mean_squared_error(y_true = t_prime[sample_idx,1],y_pred=y_pred_[:,1]))
rmse_y2

0.02007143502506732

In [22]:
# How to verify , plot predictive distribution for values from kernel 1 and kernel 2 , should be the same !
# is it called similarity verification !

## Ref:
1. http://krasserm.github.io/2018/03/19/gaussian-processes/
2. https://papers.nips.cc/paper/2420-gaussian-processes-in-reinforcement-learning.pdf

TODO
1. Deep understanding for optimization methods in <br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html